In [2]:
import os
import pandas as pd

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 27 days


In [3]:
os.system("export HIVE_OPTS='-hiveconf mapreduce.job.queuename=priority'")


def query_hive_ssh(query, file_name):
    cmd = """ssh stat1002.eqiad.wmnet "hive -e \\" """ +query+ """ \\" "> """ + file_name
    os.system(cmd)
    d = pd.read_csv(file_name,  sep='\t')
    #os.system('rm ' + file_name)
    return d

In [4]:
query = """
SELECT
tb.n as before,
ta.n as after,
(ta.n - tb.n) / tb.n as delta,
wikidata.id,
ta.page_title as page_title,
en_wikidata.en_page_title,
ta.country as country,
ta.project as project
FROM
(SELECT SUM(view_count) as n, regexp_replace(page_title, ' ', '_') as page_title, country, project
FROM wmf.pageview_hourly
WHERE year = 2015
AND month = 6
AND day BETWEEN 1 AND 8
AND country = '%(country)s'
AND project in ('%(wiki)s.wikipedia', 'en.wikipedia')
GROUP BY regexp_replace(page_title, ' ', '_'), country, project) tb
JOIN
(SELECT SUM(view_count) as n, regexp_replace(page_title, ' ', '_') as page_title, country, project
FROM wmf.pageview_hourly
WHERE year = 2015
AND month = 6
AND day BETWEEN 13 AND 20
AND country = '%(country)s'
AND project in ('%(wiki)s.wikipedia', 'en.wikipedia')
GROUP BY regexp_replace(page_title, ' ', '_'), country, project) ta
ON (ta.page_title = tb.page_title AND ta.country = tb.country AND ta.project = tb.project)
LEFT JOIN
(SELECT id, CONCAT(lang, '.wikipedia') as project, regexp_replace(page_title, ' ', '_') as page_title
FROM ellery.wikidata) wikidata
ON (ta.project = wikidata.project AND tb.project = wikidata.project AND ta.page_title = wikidata.page_title AND tb.page_title = wikidata.page_title)
LEFT JOIN
(SELECT id, regexp_replace(page_title, ' ', '_') as en_page_title
FROM ellery.wikidata
WHERE lang == 'en') en_wikidata
ON (wikidata.id == en_wikidata.id)
AND ta.n > 100
AND ta.page_title not RLIKE ':';
"""

In [5]:
params = {'wiki':'fa', 'country': 'Iran'}

In [6]:
df = query_hive_ssh(query % params, 'censorship.tsv')

/Users/ellerywulczyn/miniconda3/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [8]:
df = pd.read_csv('censorship.tsv', sep = '\t', encoding = 'utf8')

/Users/ellerywulczyn/miniconda3/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [9]:
df.shape

(726412, 8)

In [11]:
df.sort('delta', ascending = [0], inplace = True)
df.sort('delta', ascending = [0], inplace = True)

In [13]:
df.head(1000)

,before,after,delta,wikidata.id,page_title,en_wikidata.en_page_title,country,project
139862,1,1513,1512.000000,NaN,مجتبی_بزرگ_علوی,NaN,Iran,fa.wikipedia
176494,13,19631,1509.076923,Q1973619,انگور_برزیلی,Jabuticaba,Iran,fa.wikipedia
276031,1,505,504.000000,Q38358,دوبرمن_پینچر_(سگ),Doberman_Pinscher,Iran,fa.wikipedia
254767,2,1007,502.500000,Q18327968,Bijan_Allipour,Bijan_Allipour,Iran,en.wikipedia
245769,18,8706,482.666667,Q14755168,مدرسه_پیرمردها,NaN,Iran,fa.wikipedia
629077,3,792,263.000000,Q6875047,Misagh_Bahadoran,Misagh_Bahadoran,Iran,en.wikipedia
239553,4,954,237.500000,Q1147006,خوپا,Chuppah,Iran,fa.wikipedia
694371,3,660,219.000000,Q515039,ایگور_اشتیماتس,Igor_Štimac,Iran,fa.wikipedia
452672,1,212,211.000000,Q5677309,ویکی‌پدیا:یادبودهای_برگزیده/۱۴_ژوئن,NaN,Iran,fa.wikipedia
714493,123,25388,205.406504,Q7319739,رضا_قرایی,Reza_Gharaei,Iran,fa.wikipedia


In [30]:
df_en = df[df['project'] == 'en.wikipedia']

In [35]:
df[df['project'] == 'en.wikipedia'].head()

,before,after,delta,wikidata.id,page_title,en_wikidata.en_page_title,country,project
14833,6806,895705,130.605201,Q1121101,Synchronicity,Synchronicity,United States,en.wikipedia
134267,1847,218571,117.338387,Q310476,Fats_Waller,Fats_Waller,United States,en.wikipedia
43135,10247,1053684,101.828535,Q389769,The_Almighty_Johnsons,The_Almighty_Johnsons,United States,en.wikipedia
333469,1049,96119,90.629171,Q526239,Plantaris_muscle,Plantaris_muscle,United States,en.wikipedia
143192,1369,91630,65.932067,Q4928299,Blooming_onion,Blooming_onion,United States,en.wikipedia


In [36]:
df[df['project'] == 'ru.wikipedia'].head()

,before,after,delta,wikidata.id,page_title,en_wikidata.en_page_title,country,project
226616,1406,379173,268.682077,Q241382,"Фриске,_Жанна_Владимировна",Jeanna_Friske,Ukraine,ru.wikipedia
141223,1574,334822,211.720457,Q4522923,"Шепелев,_Дмитрий_Андреевич_(телеведущий)",NaN,Russia,ru.wikipedia
226617,10457,1975668,187.932581,Q241382,"Фриске,_Жанна_Владимировна",Jeanna_Friske,Russia,ru.wikipedia
226321,1127,60291,52.496894,Q2370733,"Селезнёва,_Наталья_Игоревна",Natalya_Seleznyova,Russia,ru.wikipedia
340566,1438,34669,23.109179,Q763213,Европейские_игры_2015,2015_European_Games,Ukraine,ru.wikipedia


In [37]:
df[df['project'] == 'zh.wikipedia'].head()

,before,after,delta,wikidata.id,page_title,en_wikidata.en_page_title,country,project
271896,12252,355001,27.974943,Q22502,臺灣,NaN,Taiwan,zh.wikipedia
158436,1922,51048,25.559834,Q10884109,他來自江湖,NaN,Hong Kong,zh.wikipedia
81090,1440,37925,25.336806,Q229775,布莱斯·达拉斯·霍华德,Bryce_Dallas_Howard,Taiwan,zh.wikipedia
332746,2345,44943,18.165458,Q503706,克里斯·普瑞特,Chris_Pratt,Taiwan,zh.wikipedia
261272,3635,53323,13.669326,Q15311343,端午节,NaN,Taiwan,zh.wikipedia


In [39]:
df[df['project'] == 'fa.wikipedia'].head(30)

,before,after,delta,wikidata.id,page_title,en_wikidata.en_page_title,country,project
292517,2242,20485,8.136931,Q720760,مصطفی_چمران,Mostafa_Chamran,Iran,fa.wikipedia
183172,5726,36586,5.389452,Q3267800,عملیات_کربلای_۴,Operation_Karbala-4,Iran,fa.wikipedia
324119,1753,8323,3.747861,Q317475,علی_شریعتی,Ali_Shariati,Iran,fa.wikipedia
126085,2201,10262,3.662426,Q19996727,عملیات_کربلای_۵,Siege_of_Basra,Iran,fa.wikipedia
98398,1193,4476,2.751886,Q5948961,بهروز_بقایی,NaN,Iran,fa.wikipedia
242446,3798,14152,2.726172,Q5961401,سعید_معروف,Saeid_Marouf,Iran,fa.wikipedia
194749,1029,3075,1.988338,Q5964525,مختصات_شهرهای_استان‌های_ایران_(گ-ی),NaN,Iran,fa.wikipedia
242456,1081,3207,1.966698,Q5963823,مختصات_شهرهای_استان‌های_ایران_(ف-ک),NaN,Iran,fa.wikipedia
7992,1170,3439,1.939316,NaN,شمعدونی_(سریال_تلویزیونی),NaN,Iran,fa.wikipedia
146406,1280,3617,1.825781,Q5964331,مختصات_شهرهای_استان‌های_ایران_(ت-س),NaN,Iran,fa.wikipedia


In [44]:
df[df['en_wikidata.en_page_title'] == 'Salman_Rushdie'].head(30)

,before,after,delta,wikidata.id,page_title,en_wikidata.en_page_title,country,project
140881,2655,3061,0.152919,Q44306,Salman_Rushdie,Salman_Rushdie,India,en.wikipedia
140882,9576,8065,-0.157790,Q44306,Salman_Rushdie,Salman_Rushdie,United States,en.wikipedia
